# PrimAITE Email System Request/Response Integration Demo

This notebook provides a comprehensive demonstration of the PrimAITE request/response system integration with email components. It showcases how to properly interact with email services (SMTP, POP3) and applications (Email Client) using PrimAITE's tree-based request routing system.

## Overview

The PrimAITE email system consists of three main components:

| **Component** | **Type** | **Purpose** | **Access Path** |
|:-------------:|:--------:|:-----------:|:---------------:|
| **SMTP Server** | Service | Handles email sending and relay | `["network", "node", "hostname", "service", "smtp-server", "action"]` |
| **POP3 Server** | Service | Handles email retrieval | `["network", "node", "hostname", "service", "pop3-server", "action"]` |
| **Email Client** | Application | User-facing email program | `["network", "node", "hostname", "application", "email-client", "action"]` |

## Key Learning Objectives

1. **Request System Structure**: Understanding PrimAITE's tree-based request routing
2. **Parameter Passing**: Correct method for passing parameters in requests
3. **Component Types**: Difference between Services and Applications
4. **Error Handling**: Proper error handling and response interpretation
5. **Email Operations**: Complete email workflow from sending to retrieval

## Table of Contents

- [Environment Setup](#environment-setup)
- [Network Architecture](#network-architecture)
- [SMTP Server Operations](#smtp-server-operations)
- [POP3 Server Operations](#pop3-server-operations)
- [Email Client Operations](#email-client-operations)
- [Complete Email Workflow](#complete-email-workflow)
- [Error Handling Examples](#error-handling-examples)
- [System Monitoring](#system-monitoring)
- [Summary and Best Practices](#summary-and-best-practices)

## Environment Setup

First, we initialize the PrimAITE environment and import the necessary components for our email system demonstration.

In [ ]:
# Initialize PrimAITE environment
!primaite setup

In [ ]:
# Import required PrimAITE components
from primaite.simulator.sim_container import Simulation
from primaite.simulator.network.hardware.nodes.host.computer import Computer

# Import email system components
from primaite_mail.simulator.software.smtp_server import SMTPServer
from primaite_mail.simulator.software.pop3_server import POP3Server
from primaite_mail.simulator.software.email_client import EmailClient

# Store original config to restore later
from primaite import PRIMAITE_CONFIG
original_dev_mode = PRIMAITE_CONFIG["developer_mode"]["enabled"]
original_sys_logs = PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"]
original_agent_logs = PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"]

# Enable developer mode and system logs for debugging
PRIMAITE_CONFIG["developer_mode"]["enabled"] = True
PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"] = True
PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"] = True
print("✅ All imports successful - ready to demonstrate email system integration")

## Network Architecture

We create a simple but realistic email network topology consisting of:

- **Mail Server** (`mail_server`): Hosts SMTP and POP3 services
- **Client PC** (`client_pc`): Runs the email client application

This architecture demonstrates the typical client-server email model used in enterprise environments.

In [ ]:
# Create the simulation container
sim = Simulation()
print("📊 Created PrimAITE simulation container")

# Create mail server node
# This server will host both SMTP (sending) and POP3 (retrieval) services
mail_server = Computer.from_config({
    "type": "computer",
    "hostname": "mail_server",
    "ip_address": "192.168.1.10",
    "subnet_mask": "255.255.255.0",
    "operating_state": "ON"
})

# Create client PC node
# This client will run the email client application
client_pc = Computer.from_config({
    "type": "computer",
    "hostname": "client_pc",
    "ip_address": "192.168.1.20",
    "subnet_mask": "255.255.255.0",
    "operating_state": "ON"
})

# Add nodes to the simulation network
sim.network.add_node(mail_server)
sim.network.add_node(client_pc)

print(f"🖥️  Mail server: {mail_server.config.hostname} ({mail_server.config.ip_address})")
print(f"💻 Client PC: {client_pc.config.hostname} ({client_pc.config.ip_address})")
print(f"🌐 Network contains {len(sim.network.nodes)} nodes")

### Service Installation and Configuration

Now we install the email services and applications on their respective nodes:

- **Mail Server**: Gets SMTP Server (service) and POP3 Server (service)
- **Client PC**: Gets Email Client (application)

Note the distinction between **services** (background system processes) and **applications** (user-facing programs).

In [ ]:
# Install email services on the mail server
print("📧 Installing email services on mail server...")

# SMTP Server - handles outgoing email (sending/relay)
mail_server.software_manager.install(SMTPServer)
smtp_server = mail_server.software_manager.software.get("smtp-server")
print(f"   ✅ SMTP Server: {smtp_server.name} (Port: {smtp_server.port}, State: {smtp_server.operating_state.name})")

# POP3 Server - handles incoming email retrieval
mail_server.software_manager.install(POP3Server)
pop3_server = mail_server.software_manager.software.get("pop3-server")
print(f"   ✅ POP3 Server: {pop3_server.name} (Port: {pop3_server.port}, State: {pop3_server.operating_state.name})")

# Install email client on the client PC
print("\n💻 Installing email client on client PC...")
client_pc.software_manager.install(EmailClient)
email_client = client_pc.software_manager.software.get("email-client")
print(f"   ✅ Email Client: {email_client.name} (State: {email_client.operating_state.name})")

# Share mailbox manager between SMTP and POP3 servers
# This ensures both services can access the same mailboxes
pop3_server.mailbox_manager = smtp_server.mailbox_manager
print(f"\n🔗 Linked POP3 and SMTP servers to shared mailbox system")
print(f"📊 Installation complete - {len(smtp_server.mailbox_manager.mailboxes)} mailboxes ready")

## SMTP Server Operations

The SMTP Server is a **service** that handles email sending and relay operations. As a service, it's accessed via the path:
`["network", "node", "hostname", "service", "smtp-server", "action", {parameters}]`

### Key SMTP Server Operations:

| **Operation** | **Purpose** | **Parameters** |
|:-------------:|:-----------:|:--------------:|
| `create_mailbox` | Create user mailboxes | `{"username": "string"}` |
| `list_mailboxes` | List all mailboxes | None |
| `send_message` | Send email directly | `{"sender": "string", "recipients": ["list"], "subject": "string", "body": "string"}` |
| `get_server_stats` | Get server statistics | None |
| `get_mailbox_messages` | Get messages from mailbox | `{"username": "string", "folder": "string"}` |

In [ ]:
print("=== SMTP Server Operations Demo ===")
print("\n🔧 Creating user mailboxes via request system...")

# Create mailboxes for our test users
# Note: Parameters are passed as the LAST element in the request list
test_users = ["alice", "bob", "charlie"]

for username in test_users:
    response = sim.apply_request(
        request=["network", "node", "mail_server", "service", "smtp-server", "create_mailbox", {"username": username}],
        context={}  # Context is typically empty - parameters go in the request list
    )
    
    # Check response status and provide feedback
    status_icon = "✅" if response.status == "success" else "❌"
    print(f"   {status_icon} Create mailbox for '{username}': {response.status}")
    
    if response.status == "success" and response.data:
        print(f"      📝 Response data: {response.data}")
    elif response.status != "success":
        print(f"      ⚠️  Error: {response.data.get('reason', 'Unknown error')}")

In [ ]:
# List all mailboxes to verify creation
print("\n📋 Listing all mailboxes...")
response = sim.apply_request(
    request=["network", "node", "mail_server", "service", "smtp-server", "list_mailboxes"],
    context={}  # No parameters needed for this operation
)

print(f"📊 List mailboxes result: {response.status}")
if response.status == "success":
    mailboxes = response.data.get("mailboxes", {})
    print(f"   📬 Found {len(mailboxes)} mailboxes:")
    
    for username, info in mailboxes.items():
        print(f"      👤 {username}: {info['total_messages']} messages")
        print(f"         📁 Folders: {', '.join(info['folders'])}")
else:
    print(f"   ❌ Failed to list mailboxes: {response.data}")

In [ ]:
# Send a test message directly through the SMTP server
print("\n📤 Sending test message via SMTP server...")

# Construct the email message parameters
email_params = {
    "sender": "alice@company.com",
    "recipients": ["bob@company.com", "charlie@company.com"],
    "subject": "Team Meeting Reminder",
    "body": "Hi team,\n\nDon't forget about our weekly team meeting tomorrow at 2 PM in the conference room.\n\nBest regards,\nAlice"
}

response = sim.apply_request(
    request=["network", "node", "mail_server", "service", "smtp-server", "send_message", email_params],
    context={}
)

print(f"📧 Send message result: {response.status}")
if response.status == "success":
    data = response.data
    print(f"   ✅ Successfully delivered to {data.get('delivered', 0)} recipients")
    if data.get('failed', 0) > 0:
        print(f"   ⚠️  Failed deliveries: {data.get('failed_recipients', [])}")
else:
    print(f"   ❌ Message sending failed: {response.data}")

## POP3 Server Operations

The POP3 Server is a **service** that handles email retrieval operations. Like the SMTP server, it follows the service access pattern:
`["network", "node", "hostname", "service", "pop3-server", "action", {parameters}]`

### Key POP3 Server Operations:

| **Operation** | **Purpose** | **Parameters** |
|:-------------:|:-----------:|:--------------:|
| `authenticate_user` | Verify user credentials | `{"username": "string", "password": "string"}` |
| `get_message_list` | List messages for user | `{"username": "string"}` |
| `retrieve_message` | Get specific message | `{"username": "string", "message_number": int}` |
| `delete_message` | Delete a message | `{"username": "string", "message_number": int}` |

> **💡 Best Practice**: POP3 operations typically require user authentication before accessing mailbox contents.

In [ ]:
print("=== POP3 Server Operations Demo ===")
print("\n🔐 Authenticating user for POP3 access...")

# Authenticate a user for POP3 access
# In a real system, this would verify against a user database
auth_params = {
    "username": "bob",
    "password": "password123"  # Simple password for demonstration
}

response = sim.apply_request(
    request=["network", "node", "mail_server", "service", "pop3-server", "authenticate_user", auth_params],
    context={}
)

print(f"🔑 Authentication result: {response.status}")
if response.status == "success":
    auth_data = response.data
    print(f"   ✅ User '{auth_data['username']}' authenticated successfully")
    print(f"   📬 Available messages: {auth_data.get('message_count', 0)}")
else:
    print(f"   ❌ Authentication failed: {response.data.get('reason', 'Unknown error')}")

In [ ]:
# Get message list for the authenticated user
print("\n📋 Getting message list for user...")

response = sim.apply_request(
    request=["network", "node", "mail_server", "service", "pop3-server", "get_message_list", {"username": "bob"}],
    context={}
)

print(f"📨 Message list result: {response.status}")
if response.status == "success":
    data = response.data
    print(f"   👤 User: {data['username']}")
    print(f"   📊 Message count: {data['message_count']}")
    print(f"   💾 Total size: {data['total_size']} bytes")
    
    if data['message_count'] > 0:
        print("   📧 Messages:")
        for msg in data['messages']:
            print(f"      {msg['number']}: {msg['subject']} ({msg['size']} bytes)")
            print(f"         From: {msg['sender']}")
    else:
        print("   📭 No messages found")
else:
    print(f"   ❌ Failed to get message list: {response.data}")

## Email Client Operations

The Email Client is an **application** (user-facing program) rather than a service. This is a crucial distinction in PrimAITE:

- **Services**: Background system processes (SMTP Server, POP3 Server)
- **Applications**: User-facing programs (Email Client, Web Browser)

Applications use a different access path:
`["network", "node", "hostname", "application", "email-client", "action", {parameters}]`

### Key Email Client Operations:

| **Operation** | **Purpose** | **Parameters** |
|:-------------:|:-----------:|:--------------:|
| `configure` | Set client configuration | `{"username": "string", "smtp_server": "string", "pop3_server": "string"}` |
| `show_status` | Display client status | None |
| `send_email` | Send email via SMTP | `{"to": ["list"], "subject": "string", "body": "string", "from": "string"}` |
| `test_connection` | Test server connectivity | `{"server_type": "smtp/pop3"}` |

> **⚠️ Important**: Note the path difference - `"application"` instead of `"service"`

In [ ]:
print("=== Email Client Operations Demo ===")
print("\n⚙️ Configuring email client...")

# Configure the email client with server settings
# This is similar to setting up Outlook or Thunderbird
client_config = {
    "username": "alice@company.com",
    "smtp_server": "192.168.1.10",  # Our mail server IP
    "pop3_server": "192.168.1.10",  # Same server for POP3
    "password": "password123"
}

response = sim.apply_request(
    request=["network", "node", "client_pc", "application", "email-client", "configure", client_config],
    context={}
)

print(f"🔧 Configuration result: {response.status}")
if response.status == "success":
    print(f"   ✅ Email client configured successfully")
    print(f"   📝 Response: {response.data.get('message', 'Configuration applied')}")
else:
    print(f"   ❌ Configuration failed: {response.data}")

## Error Handling Examples

Understanding error conditions is crucial for robust email system integration. This section demonstrates common error scenarios and their proper handling.

### Common Error Types:

| **Error Type** | **Status** | **Cause** | **Example** |
|:--------------:|:----------:|:---------:|:-----------:|
| **Unreachable** | `unreachable` | Component doesn't exist | Wrong hostname or service name |
| **Failure** | `failure` | Operation failed | Invalid parameters or system error |
| **Success** | `success` | Operation completed | Normal successful operation |

> **💡 Debugging Tip**: Always check both `response.status` and `response.data` for complete error information.

In [ ]:
print("=== Error Handling Examples ===")
print("\n🚫 Demonstrating common error scenarios...")

# Error 1: Accessing non-existent mailbox
print("\n1️⃣ Attempting to access non-existent mailbox...")
response = sim.apply_request(
    request=["network", "node", "mail_server", "service", "smtp-server", "get_mailbox_messages", {
        "username": "nonexistent_user"
    }],
    context={}
)

print(f"   📊 Result: {response.status}")
if response.status == "failure":
    print(f"   ✅ Correctly handled non-existent mailbox")
    print(f"   📝 Error reason: {response.data.get('reason', 'Unknown')}")
else:
    print(f"   ⚠️ Unexpected result: {response.data}")

# Error 2: Wrong component type access
print("\n2️⃣ Attempting to access email client as service (wrong path)...")
response = sim.apply_request(
    request=["network", "node", "client_pc", "service", "email-client", "show_status"],  # Wrong: should be "application"
    context={}
)

print(f"   📊 Result: {response.status}")
if response.status == "unreachable":
    print(f"   ✅ Correctly rejected wrong component type access")
    print(f"   💡 Remember: Email client is an APPLICATION, not a service")
else:
    print(f"   ⚠️ Unexpected result for wrong component type")

## Summary and Best Practices

This comprehensive demonstration has covered the essential aspects of PrimAITE's email system integration using the request/response system. Here are the key takeaways and best practices for effective email system development.

### Key Concepts Learned

1. **Request System Structure**: PrimAITE uses tree-based navigation with relative paths
2. **Component Types**: Services (background) vs Applications (user-facing) have different access patterns
3. **Parameter Passing**: Parameters go in the request list as the last element, not in context
4. **Error Handling**: Always check both status and data for complete error information
5. **System Integration**: Components work together through shared resources (mailbox manager)

### Best Practices Summary

| **Category** | **Best Practice** | **Example** |
|:------------:|:----------------:|:-----------:|
| **Request Paths** | Use correct component type | `"application"` for email client, `"service"` for servers |
| **Parameters** | Pass in request list | `["action", {"param": "value"}]` |
| **Error Handling** | Check status and data | `if response.status == "success":` |
| **Monitoring** | Regular health checks | Use `get_server_stats` and `show_status` |
| **Integration** | Share resources properly | Link mailbox managers between services |

### Common Pitfalls to Avoid

- ❌ **Wrong Component Type**: Accessing applications as services or vice versa
- ❌ **Parameter Location**: Putting parameters in context instead of request list
- ❌ **Missing Error Handling**: Not checking response status before using data
- ❌ **Incorrect Paths**: Using absolute paths instead of relative component navigation
- ❌ **Resource Isolation**: Not sharing resources between related components

### Next Steps

To continue developing with the PrimAITE email system:

1. **Explore Advanced Features**: Investigate additional email protocols (IMAP, Exchange)
2. **Security Integration**: Add authentication and encryption capabilities
3. **Performance Optimization**: Implement caching and connection pooling
4. **Monitoring Enhancement**: Create comprehensive logging and alerting systems
5. **Agent Integration**: Develop AI agents that can interact with the email system

### Documentation References

- **PrimAITE Request System**: See steering document `primaite-request-system.md`
- **Component Architecture**: Review PrimAITE core documentation
- **Email Protocol Standards**: RFC 5321 (SMTP), RFC 1939 (POP3)
- **Testing Patterns**: Examine test files in `tests/` directory

---

**🎉 Congratulations!** You have successfully completed the PrimAITE Email System Request/Response Integration demonstration. You now have the knowledge and tools to build sophisticated email-based simulations and AI training environments.

In [ ]:
# Returning to the original developer mode configuration.
PRIMAITE_CONFIG["developer_mode"]["enabled"] = original_dev_mode
PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"] = original_sys_logs
PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"] = original_agent_logs